In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import glob
import gzip
import json
from datetime import datetime
        
root_dir = "/Users/nshelton/Library/Mobile Documents/iCloud~com~bigpaua~LearnerCoacher/Documents/Export/JSON/Daily"

compressed_files = glob.glob(f"{root_dir}/*.json.gz")

points = []
times = []

for compressed_file in compressed_files:

  # if "2024-09-09.json.gz" not in compressed_file: continue
  # compressed_file = compressed_files[21]
  day = compressed_file.split('/')[-1]

  with gzip.open(compressed_file, 'rb') as f:
    data = json.loads(f.read().decode('utf-8'))

  timeline = data['timelineItems']
  samples = []

  for item in timeline:
      samples += item['samples']

  print(f"{day} : {len(samples)} samples")


  these_points = []
  these_times = []
  for s in samples:
      if 'location' in s:
        location = s['location']
        if location is not None and 'latitude' in location:
          these_times.append(location['timestamp'])
          these_points.append([location['latitude'], location['longitude']])

  # points += these_points
  # times += these_times

  points = np.array(these_points)
  datetimes = [datetime.strptime(time_string, '%Y-%m-%dT%H:%M:%SZ') for time_string in these_times]
  timestamps = [dt.timestamp() for dt in datetimes]


  from sklearn.cluster import DBSCAN

  # Perform clustering to identify points of interest
  epsilon = 0.0001  # Adjust this value based on the scale of your data
  min_samples = 20   # Minimum number of points to form a cluster
  dbscan = DBSCAN(eps=epsilon, min_samples=min_samples).fit(points)


  # Plot clustered data
  plt.figure(figsize=(12, 8))
  plt.scatter(points[:, 1], points[:, 0], c=dbscan.labels_, cmap='viridis', alpha=0.5, s=3)
  plt.title(compressed_file)
  plt.xlabel('Longitude')
  plt.ylabel('Latitude')
  plt.colorbar(label='Cluster Label')


  # Identify cluster centers
  unique_labels = set(dbscan.labels_)
  cluster_centers = []

  for label in unique_labels:
      if label != -1:  # -1 represents noise in DBSCAN
          cluster_points = points[dbscan.labels_ == label]
          center = np.mean(cluster_points, axis=0)
          cluster_centers.append((center[0], center[1]))

  cluster_centers = np.array(cluster_centers)
  plt.scatter(cluster_centers[:, 1], cluster_centers[:, 0], alpha=0.5, s=300)


In [ ]:
plt.scatter(timestamps, dbscan.labels_)

In [ ]:
import folium

# Create a map centered at a specific location
m = folium.Map(location=points.mean(axis=0), zoom_start=13)

print(len(cluster_centers))
# Sample data: list of (lat, lon) pairs
# locations = points[::100]
locations = cluster_centers

# Add markers to the map
for (lat, lon), time in zip(locations, times):
    folium.Marker([lat, lon], popup=time).add_to(m)

# Display the map in the notebook
m

In [7]:
import sqlite3

def initialize_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS locations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            time TEXT,
            lat Decimal(8,6),
            lng Decimal(9,6)
        )
    ''')
    conn.commit()
    return conn

db_path = 'locations.db'
conn = initialize_database(db_path)

cursor = conn.cursor()
for sample in samples:
    location = sample['location']

    if location == None:
        continue
    # print(location)
    cursor.execute('''
      INSERT INTO locations (time, lat, lng)
      VALUES (?, ?, ?)
      ''', (location['timestamp'], location['latitude'], location['longitude']))
conn.commit() 


